# Calculate noise and dice for Total Segmentator and Vista3d

In [1]:
config = {}
config['images'] = '/processed/Public/TCIA'
config['gt'] = '/processed/Public/TCIA/labels-vista3d'
config['predictions'] = '/processed/Public/TCIA/predictions/vista3d'
config['predictions_TS'] = '/processed/Public/TCIA/predictions/TotalSegmentator'
config['labels_json'] = 'labels-TCIA-pediatric.json'
config['labels_json_TS'] = 'labels-TCIA-pediatric-TS.json'

In [2]:
# !pip install pyradiomics

In [3]:
import json
import tqdm
import pandas as pd
with open(config['labels_json'],'r') as jsonfile:
    config['labels'] = json.load(jsonfile)

In [4]:
from radiomics import featureextractor

In [5]:
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import laplace
from radiomics import featureextractor
import os 

def estimate_noise_mad(volume):
    """
    Estimate noise using Median Absolute Deviation (MAD).
    """
    pixels = volume.flatten()
    median_val = np.median(pixels)
    mad = np.median(np.abs(pixels - median_val))
    sigma_est = 1.4826 * mad
    return sigma_est

def estimate_noise_laplacian(volume):
    """
    Estimate noise using the Laplacian operator.
    """
    laplacian_vol = laplace(volume)
    # Mean absolute value of the Laplacian can correlate with noise level
    return np.mean(np.abs(laplacian_vol))

# Calculate boundaries for the central 90%
# Remove 5% on each side for each dimension
def central_bounds(dim_size, fraction=0.05):
    start = int(dim_size * fraction)
    end = int(dim_size * (1 - fraction))
    return start, end

def estimate_noise_pyradiomics(image, mask_path=None):
    """
    Use PyRadiomics to extract a feature as a proxy for noise estimation.
    If no mask is provided, use the entire image as ROI.
    """
    # Initialize radiomics feature extractor
    extractor = featureextractor.RadiomicsFeatureExtractor()

    # Create a full-volume mask if none provided
    if mask_path:
        mask = sitk.ReadImage(mask_path)
    else:
        # Create an array of ones with the same shape as the image
        array = sitk.GetArrayFromImage(image)
        shape = array.shape
        # For 3D images, shape = (depth, height, width)
        # For 2D images, shape = (height, width)
        # We handle both cases generically.
        if len(shape) == 3:
            z_start, z_end = central_bounds(shape[0])
            y_start, y_end = central_bounds(shape[1])
            x_start, x_end = central_bounds(shape[2])
        elif len(shape) == 2:
            y_start, y_end = central_bounds(shape[0])
            x_start, x_end = central_bounds(shape[1])
        else:
            raise ValueError("Image dimensionality not supported.")

        # Create a mask array of zeros with the same shape as the image
        mask_array = np.zeros_like(array, dtype=np.uint8)

        # Set ones for the central 90% region
        if len(shape) == 3:
            mask_array[z_start:z_end, y_start:y_end, x_start:x_end] = 1
        elif len(shape) == 2:
            mask_array[y_start:y_end, x_start:x_end] = 1
        mask = sitk.GetImageFromArray(mask_array)
        mask.CopyInformation(image)

    # Extract features using PyRadiomics
    result = extractor.execute(image, mask)

    # Use RootMeanSquared as a proxy feature related to intensity variation (noise)
    # (Replace with a more appropriate feature or process as needed)
    noise_feature = result.get('original_firstorder_RootMeanSquared', None)
    return noise_feature

In [6]:
import SimpleITK as sitk
import numpy as np

def sitk_to_numpy(image):
    """Convert a SimpleITK image to a numpy array."""
    return sitk.GetArrayFromImage(image)

def dice_coefficient(true_array, pred_array, label):
    """
    Calculate Dice coefficient for a specific label.
    Also return intersection and sum of sizes for micro-average computation.
    """
    true_bin = (true_array == label)
    pred_bin = (pred_array == label)

    intersection = np.logical_and(true_bin, pred_bin).sum()
    sum_sizes = true_bin.sum() + pred_bin.sum()

    # Handle case of no voxels for label in both images
    dice = np.nan if sum_sizes == 0 else 2.0 * intersection / sum_sizes
    return dice, intersection, sum_sizes

def multi_class_dice(true_image, pred_image, labels):
    """
    Compute label-wise, macro-average, and micro-average Dice for multiple labels.
    
    Parameters:
        true_image (sitk.Image): Ground truth segmentation.
        pred_image (sitk.Image): Predicted segmentation.
        labels (list or iterable): List of label values to compute Dice for.
        
    Returns:
        dict: Dice scores for each label.
        float: Macro-average Dice score.
        float: Micro-average Dice score.
    """
    true_array = sitk_to_numpy(true_image)
    pred_array = sitk_to_numpy(pred_image)

    dice_scores = {}
    macro_sum = 0.0

    # For micro-average calculations
    total_intersection = 0
    total_sum_sizes = 0

    for key,label in labels.items():
        dice, intersection, sum_sizes = dice_coefficient(true_array, pred_array, label)
        dice_scores[key] = dice
        macro_sum += dice

        total_intersection += intersection
        total_sum_sizes += sum_sizes

    macro_average_dice = macro_sum / len(labels) if labels else float('nan')
    # Handle micro-average division by zero
    micro_average_dice = 1.0 if total_sum_sizes == 0 else 2.0 * total_intersection / total_sum_sizes

    return dice_scores, macro_average_dice, micro_average_dice


In [7]:
# Make sure label exists
ct_images = [f for f in os.listdir(config['gt']) if f.endswith('.nii.gz')]

results = []
for ct_name in tqdm.tqdm(ct_images):
    try:
        ct_image_path = os.path.join(config['images'],ct_name)
        ct_image = sitk.ReadImage(ct_image_path)
        ct_volume = sitk.GetArrayFromImage(ct_image)

        # Calculate noise estimates.
        mad_noise = estimate_noise_mad(ct_volume)
        laplace_noise = estimate_noise_laplacian(ct_volume)
        pyradiomics_noise = 0 #estimate_noise_pyradiomics(ct_image)

        label_path = os.path.join(config['gt'],ct_name)
        pred_path = os.path.join(config['predictions'],ct_name)
        label_image = sitk.ReadImage(label_path)
        pred_image = sitk.ReadImage(pred_path)

        dice_scores, macro_avg, micro_avg = multi_class_dice(label_image, pred_image, config['labels'])

        # Prepare record for current CT
        record = {
            'ct_name': ct_name,
            'mad_noise': mad_noise,
            'laplace_noise': laplace_noise,
            'pyradiomics_noise': pyradiomics_noise,
            'macro_avg_dice': macro_avg,
            'micro_avg_dice': micro_avg
        }

        # Add organ-specific Dice scores to the record
        for organ, dice_val in dice_scores.items():
            record[f'dice_{organ}'] = dice_val

        results.append(record)
    except Exception as e:
        print(e)

100% 358/358 [1:57:24<00:00, 19.68s/it]  


In [9]:
# Create a DataFrame from results and save to CSV
df = pd.DataFrame(results)
output_csv_path = "ct_noise_dice_results.csv"  # Specify desired output path
df.to_csv(output_csv_path, index=False)

print(f"Results saved to {output_csv_path}")

Results saved to ct_noise_dice_results.csv
